 Current Best Performance:
- Simple LSTM: Magnitude ±0.2 = 87.5%, Frequency range = 12.7%
- Attention LSTM: Magnitude ±0.2 = 50.0%, Frequency range = 38.0%

In [ ]:
# Install required packages if needed
!pip install torch torchvision torchaudio
!pip install pandas numpy matplotlib seaborn
!pip install scikit-learn optuna

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import logging
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Add src to path for model imports
import sys
sys.path.append('src')

# Import your models
try:
    from models.shared_lstm_model import SharedLSTMModel, WeightedEarthquakeLoss
    from models.attention_shared_lstm_model import AttentionSharedLSTMModel
    from models.shared_lstm_trainer import SharedLSTMTrainer
    from models.attention_shared_lstm_trainer import AttentionSharedLSTMTrainer
    print("✅ Models imported successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure you're running this notebook from the project root directory")

## 🎯 Hyperparameter Configuration

In [ ]:
# Current Best Configuration (Baseline)
BASELINE_CONFIG = {
    # Model Architecture
    'input_seq_features': 12,
    'metadata_features': 4,
    'lookback_years': 10,
    'lstm_hidden_1': 64,
    'lstm_hidden_2': 32,
    'dense_hidden': 32,
    'dropout_rate': 0.25,
    'freq_head_type': 'linear',
    
    # Training Parameters
    'learning_rate': 5e-4,
    'weight_decay': 1e-4,
    'num_epochs': 300,
    'patience': 15,
    'batch_size': 32,
    
    # Loss Weights
    'magnitude_weight': 2.0,
    'frequency_weight': 1.0,
    'correlation_weight': 0.0,
    
    # Frequency Scaling
    'frequency_scale_init': 10.0,
    'frequency_bias_init': 2.0,
    'scaling_lr_multiplier': 20,
    'scaling_wd_multiplier': 3,
    
    # Training Stability
    'gradient_clip': 0.5,
    'scheduler_T0': 15,
    'scheduler_T_mult': 2
}

print("📋 Baseline Configuration:")
for key, value in BASELINE_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# Hyperparameter Search Spaces
TUNING_SPACES = {
    # Priority 1: Fix Frequency Range Compression
    'frequency_scaling': {
        'frequency_scale_init': [5.0, 10.0, 15.0, 20.0, 25.0],
        'frequency_bias_init': [1.0, 2.0, 3.0, 5.0, 7.0],
        'scaling_lr_multiplier': [15, 20, 25, 30, 35],
        'scaling_wd_multiplier': [2, 3, 4, 5, 6]
    },
    
    # Priority 2: Balance Loss Weights
    'loss_weights': {
        'magnitude_weight': [1.0, 1.5, 2.0, 2.5, 3.0],
        'frequency_weight': [0.8, 1.0, 1.2, 1.5, 2.0]
    },
    
    # Priority 3: Learning Rate Tuning
    'learning_rates': {
        'learning_rate': [3e-4, 5e-4, 7e-4, 1e-3, 1.5e-3],
        'weight_decay': [5e-5, 1e-4, 2e-4, 5e-4]
    },
    
    # Priority 4: Architecture Tuning
    'architecture': {
        'lstm_hidden_1': [48, 64, 80, 96],
        'lstm_hidden_2': [24, 32, 40, 48],
        'dense_hidden': [24, 32, 40, 48],
        'dropout_rate': [0.15, 0.25, 0.35, 0.45]
    },
    
    # Priority 5: Training Stability
    'training_stability': {
        'patience': [10, 15, 20, 25, 30],
        'gradient_clip': [0.3, 0.5, 0.7, 1.0],
        'scheduler_T0': [10, 15, 20, 25]
    }
}

print("🔍 Tuning Search Spaces:")
for category, params in TUNING_SPACES.items():
    print(f"\n{category.upper()}:")
    for param, values in params.items():
        print(f"  {param}: {values}")

## 🧪 Manual Hyperparameter Testing

In [ ]:
class HyperparameterTester:
    """Class to test different hyperparameter combinations"""
    
    def __init__(self, base_config: Dict):
        self.base_config = base_config.copy()
        self.results = []
        
    def create_model(self, config: Dict, model_type: str = 'simple') -> nn.Module:
        """Create model with given configuration"""
        if model_type == 'simple':
            model = SharedLSTMModel(
                input_seq_features=config['input_seq_features'],
                metadata_features=config['metadata_features'],
                lookback_years=config['lookback_years'],
                lstm_hidden_1=config['lstm_hidden_1'],
                lstm_hidden_2=config['lstm_hidden_2'],
                dense_hidden=config['dense_hidden'],
                dropout_rate=config['dropout_rate'],
                freq_head_type=config['freq_head_type']
            )
        else:  # attention
            model = AttentionSharedLSTMModel(
                input_seq_features=config['input_seq_features'],
                metadata_features=config['metadata_features'],
                lookback_years=config['lookback_years'],
                lstm_hidden_1=config['lstm_hidden_1'],
                lstm_hidden_2=config['lstm_hidden_2'],
                dense_hidden=config['dense_hidden'],
                dropout_rate=config['dropout_rate'],
                freq_head_type=config['freq_head_type']
            )
        
        # Set custom frequency scaling parameters
        if hasattr(model, 'frequency_scale'):
            model.frequency_scale.data.fill_(config['frequency_scale_init'])
        if hasattr(model, 'frequency_bias'):
            model.frequency_bias.data.fill_(config['frequency_bias_init'])
            
        return model
    
    def test_configuration(self, config: Dict, model_type: str = 'simple') -> Dict:
        """Test a single configuration"""
        print(f"\n🧪 Testing {model_type.upper()} LSTM with config:")
        for key, value in config.items():
            if key in self.base_config and value != self.base_config[key]:
                print(f"  {key}: {self.base_config[key]} → {value}")
        
        try:
            # Create model
            model = self.create_model(config, model_type)
            
            # Create dummy data for testing
            batch_size = 4
            input_sequence = torch.randn(batch_size, config['lookback_years'], config['input_seq_features'])
            metadata = torch.randn(batch_size, config['metadata_features'])
            
            # Test forward pass
            with torch.no_grad():
                magnitude_pred, frequency_pred = model(input_sequence, metadata)
            
            # Calculate prediction ranges
            mag_range = magnitude_pred.max().item() - magnitude_pred.min().item()
            freq_range = frequency_pred.max().item() - frequency_pred.min().item()
            
            # Count parameters
            total_params = sum(p.numel() for p in model.parameters())
            
            result = {
                'config': config.copy(),
                'model_type': model_type,
                'magnitude_range': mag_range,
                'frequency_range': freq_range,
                'total_params': total_params,
                'status': 'success'
            }
            
            print(f"✅ Test successful!")
            print(f"  Magnitude range: {mag_range:.4f}")
            print(f"  Frequency range: {freq_range:.4f}")
            print(f"  Total parameters: {total_params:,}")
            
        except Exception as e:
            result = {
                'config': config.copy(),
                'model_type': model_type,
                'error': str(e),
                'status': 'failed'
            }
            print(f"❌ Test failed: {e}")
        
        self.results.append(result)
        return result
    
    def run_grid_search(self, param_grid: Dict, model_type: str = 'simple') -> List[Dict]:
        """Run grid search over parameter combinations"""
        print(f"\n🚀 Starting grid search for {model_type.upper()} LSTM...")
        
        # Generate all combinations
        import itertools
        keys = list(param_grid.keys())
        values = list(param_grid.values())
        combinations = list(itertools.product(*values))
        
        print(f"Total combinations to test: {len(combinations)}")
        
        for i, combination in enumerate(combinations):
            config = self.base_config.copy()
            for key, value in zip(keys, combination):
                config[key] = value
            
            print(f"\n--- Test {i+1}/{len(combinations)} ---")
            self.test_configuration(config, model_type)
        
        return self.results
    
    def get_best_configs(self, metric: str = 'frequency_range', top_k: int = 5) -> List[Dict]:
        """Get top-k configurations based on a metric"""
        successful_results = [r for r in self.results if r['status'] == 'success']
        
        if not successful_results:
            return []
        
        # Sort by metric (higher is better for ranges)
        sorted_results = sorted(successful_results, key=lambda x: x[metric], reverse=True)
        
        return sorted_results[:top_k]
    
    def plot_results(self):
        """Plot results summary"""
        successful_results = [r for r in self.results if r['status'] == 'success']
        
        if not successful_results:
            print("No successful results to plot")
            return
        
        # Create DataFrame for plotting
        df = pd.DataFrame(successful_results)
        
        # Plot frequency range vs magnitude range
        plt.figure(figsize=(12, 8))
        
        plt.subplot(2, 2, 1)
        plt.scatter(df['frequency_range'], df['magnitude_range'], alpha=0.7)
        plt.xlabel('Frequency Range')
        plt.ylabel('Magnitude Range')
        plt.title('Frequency vs Magnitude Range')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 2)
        plt.hist(df['frequency_range'], bins=20, alpha=0.7, edgecolor='black')
        plt.xlabel('Frequency Range')
        plt.ylabel('Count')
        plt.title('Frequency Range Distribution')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 3)
        plt.hist(df['magnitude_range'], bins=20, alpha=0.7, edgecolor='black')
        plt.xlabel('Magnitude Range')
        plt.ylabel('Count')
        plt.title('Magnitude Range Distribution')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 4)
        plt.scatter(df['total_params'], df['frequency_range'], alpha=0.7)
        plt.xlabel('Total Parameters')
        plt.ylabel('Frequency Range')
        plt.title('Parameters vs Frequency Range')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Show top configurations
        print("\n🏆 Top 5 Configurations by Frequency Range:")
        top_configs = self.get_best_configs('frequency_range', 5)
        for i, config in enumerate(top_configs):
            print(f"\n{i+1}. Frequency Range: {config['frequency_range']:.4f}")
            print(f"   Magnitude Range: {config['magnitude_range']:.4f}")
            print(f"   Model Type: {config['model_type']}")
            print(f"   Key Changes:")
            for key, value in config['config'].items():
                if key in self.base_config and value != self.base_config[key]:
                    print(f"     {key}: {self.base_config[key]} → {value}")

## 🧪 Test 1: Frequency Scaling Parameters (Priority 1)

In [ ]:
# Initialize tester
tester = HyperparameterTester(BASELINE_CONFIG)

# Test frequency scaling parameters
print("🎯 Testing Frequency Scaling Parameters (Priority 1)")
print("Goal: Increase frequency prediction range from 12.7% to 40-60%")

# Test simple LSTM first
frequency_scaling_results = tester.run_grid_search(
    TUNING_SPACES['frequency_scaling'], 
    model_type='simple'
)

# Show results
print("\n📊 Frequency Scaling Results:")
tester.plot_results()

## 🧪 Test 2: Loss Weight Balancing (Priority 2)

In [ ]:
# Test loss weight combinations
print("🎯 Testing Loss Weight Balancing (Priority 2)")
print("Goal: Balance magnitude vs frequency performance")

# Use best frequency scaling config from previous test
best_freq_config = tester.get_best_configs('frequency_range', 1)[0]
print(f"\nUsing best frequency scaling config:")
print(f"Frequency range: {best_freq_config['frequency_range']:.4f}")

# Update base config with best frequency scaling
updated_config = BASELINE_CONFIG.copy()
for key, value in best_freq_config['config'].items():
    if key in TUNING_SPACES['frequency_scaling']:
        updated_config[key] = value

# Create new tester with updated config
tester2 = HyperparameterTester(updated_config)

# Test loss weights
loss_weight_results = tester2.run_grid_search(
    TUNING_SPACES['loss_weights'], 
    model_type='simple'
)

# Show results
print("\n📊 Loss Weight Results:")
tester2.plot_results()

## 🧪 Test 3: Learning Rate Optimization (Priority 3)

In [ ]:
# Test learning rates
print("🎯 Testing Learning Rate Optimization (Priority 3)")
print("Goal: Find optimal learning rate for training stability")

# Use best config from previous tests
best_overall_config = tester2.get_best_configs('frequency_range', 1)[0]
print(f"\nUsing best overall config:")
print(f"Frequency range: {best_overall_config['frequency_range']:.4f}")
print(f"Magnitude range: {best_overall_config['magnitude_range']:.4f}")

# Update config with best parameters
final_config = updated_config.copy()
for key, value in best_overall_config['config'].items():
    if key in TUNING_SPACES['loss_weights']:
        final_config[key] = value

# Create final tester
tester3 = HyperparameterTester(final_config)

# Test learning rates
lr_results = tester3.run_grid_search(
    TUNING_SPACES['learning_rates'], 
    model_type='simple'
)

# Show results
print("\n📊 Learning Rate Results:")
tester3.plot_results()

## 🧪 Test 4: Attention Model Tuning

In [ ]:
# Test attention model with best parameters
print("🎯 Testing Attention Model with Best Parameters")
print("Goal: See if attention model can match simple LSTM performance")

# Get best config from all tests
best_config = tester3.get_best_configs('frequency_range', 1)[0]
print(f"\nBest config from simple LSTM:")
print(f"Frequency range: {best_config['frequency_range']:.4f}")
print(f"Magnitude range: {best_config['magnitude_range']:.4f}")

# Test attention model with same config
attention_tester = HyperparameterTester(best_config['config'])
attention_result = attention_tester.test_configuration(
    best_config['config'], 
    model_type='attention'
)

# Compare results
print("\n📊 Simple LSTM vs Attention LSTM Comparison:")
print(f"Simple LSTM:")
print(f"  Frequency range: {best_config['frequency_range']:.4f}")
print(f"  Magnitude range: {best_config['magnitude_range']:.4f}")
print(f"  Total parameters: {best_config['total_params']:,}")

print(f"\nAttention LSTM:")
print(f"  Frequency range: {attention_result['frequency_range']:.4f}")
print(f"  Magnitude range: {attention_result['magnitude_range']:.4f}")
print(f"  Total parameters: {attention_result['total_params']:,}")

# Calculate improvements
freq_improvement = ((attention_result['frequency_range'] - best_config['frequency_range']) / best_config['frequency_range']) * 100
mag_improvement = ((attention_result['magnitude_range'] - best_config['magnitude_range']) / best_config['magnitude_range']) * 100

print(f"\n📈 Improvements with Attention:")
print(f"  Frequency range: {freq_improvement:+.1f}%")
print(f"  Magnitude range: {mag_improvement:+.1f}%")

## 🎯 Final Recommendations

In [ ]:
def generate_recommendations():
    """Generate final hyperparameter recommendations"""
    print("🎯 FINAL HYPERPARAMETER RECOMMENDATIONS")
    print("=" * 60)
    
    # Get best configs from all testers
    all_results = []
    if hasattr(tester, 'results'):
        all_results.extend(tester.results)
    if hasattr(tester2, 'results'):
        all_results.extend(tester2.results)
    if hasattr(tester3, 'results'):
        all_results.extend(tester3.results)
    
    successful_results = [r for r in all_results if r['status'] == 'success']
    
    if not successful_results:
        print("❌ No successful results to analyze")
        return
    
    # Find best configurations
    best_freq = max(successful_results, key=lambda x: x['frequency_range'])
    best_mag = max(successful_results, key=lambda x: x['magnitude_range'])
    best_balanced = max(successful_results, key=lambda x: x['frequency_range'] + x['magnitude_range'])
    
    print("\n🏆 BEST FREQUENCY PREDICTION:")
    print(f"  Frequency range: {best_freq['frequency_range']:.4f}")
    print(f"  Magnitude range: {best_freq['magnitude_range']:.4f}")
    print(f"  Model type: {best_freq['model_type']}")
    print("  Key parameters:")
    for key, value in best_freq['config'].items():
        if key in BASELINE_CONFIG and value != BASELINE_CONFIG[key]:
            print(f"    {key}: {BASELINE_CONFIG[key]} → {value}")
    
    print("\n🏆 BEST MAGNITUDE PREDICTION:")
    print(f"  Frequency range: {best_mag['frequency_range']:.4f}")
    print(f"  Magnitude range: {best_mag['magnitude_range']:.4f}")
    print(f"  Model type: {best_mag['model_type']}")
    print("  Key parameters:")
    for key, value in best_mag['config'].items():
        if key in BASELINE_CONFIG and value != BASELINE_CONFIG[key]:
            print(f"    {key}: {BASELINE_CONFIG[key]} → {value}")
    
    print("\n🏆 BEST BALANCED PERFORMANCE:")
    print(f"  Frequency range: {best_balanced['frequency_range']:.4f}")
    print(f"  Magnitude range: {best_balanced['magnitude_range']:.4f}")
    print(f"  Model type: {best_balanced['model_type']}")
    print("  Key parameters:")
    for key, value in best_balanced['config'].items():
        if key in BASELINE_CONFIG and value != BASELINE_CONFIG[key]:
            print(f"    {key}: {BASELINE_CONFIG[key]} → {value}")
    
    # Generate config files
    print("\n💾 GENERATING CONFIGURATION FILES:")
    
    configs = {
        'best_frequency.json': best_freq['config'],
        'best_magnitude.json': best_mag['config'],
        'best_balanced.json': best_balanced['config']
    }
    
    for filename, config in configs.items():
        with open(filename, 'w') as f:
            json.dump(config, f, indent=2)
        print(f"  ✅ {filename} saved")
    
    print("\n🚀 NEXT STEPS:")
    print("  1. Use the best configurations in your main training script")
    print("  2. Retrain models with optimized parameters")
    print("  3. Compare final performance on test set")
    print("  4. Consider ensemble approach for production")

# Generate recommendations
generate_recommendations()

## 🚀 Advanced Tuning with Optuna (Optional)

In [ ]:
# Advanced hyperparameter optimization with Optuna
try:
    import optuna
    
    def objective(trial):
        """Optuna objective function for hyperparameter optimization"""
        
        # Suggest hyperparameters
        config = {
            'frequency_scale_init': trial.suggest_float('freq_scale', 5.0, 30.0),
            'frequency_bias_init': trial.suggest_float('freq_bias', 1.0, 10.0),
            'magnitude_weight': trial.suggest_float('mag_weight', 1.0, 4.0),
            'frequency_weight': trial.suggest_float('freq_weight', 0.5, 3.0),
            'learning_rate': trial.suggest_float('lr', 1e-4, 2e-3, log=True),
            'dropout_rate': trial.suggest_float('dropout', 0.1, 0.5),
            'lstm_hidden_1': trial.suggest_categorical('lstm1', [48, 64, 80, 96]),
            'lstm_hidden_2': trial.suggest_categorical('lstm2', [24, 32, 40, 48])
        }
        
        try:
            # Test configuration
            test_tester = HyperparameterTester(BASELINE_CONFIG)
            result = test_tester.test_configuration(config, 'simple')
            
            if result['status'] == 'success':
                # Optimize for frequency range (primary) and magnitude range (secondary)
                score = result['frequency_range'] * 0.7 + result['magnitude_range'] * 0.3
                return score
            else:
                return -1000  # Penalty for failed configurations
                
        except Exception as e:
            return -1000  # Penalty for errors
    
    # Create study
    study = optuna.create_study(direction='maximize')
    
    print("🚀 Starting Optuna optimization...")
    print("This will take some time but will find optimal parameters automatically!")
    
    # Run optimization
    study.optimize(objective, n_trials=50, timeout=300)  # 50 trials or 5 minutes
    
    print("\n🏆 Optuna Optimization Results:")
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best value: {study.best_trial.value:.4f}")
    print(f"Best params: {study.best_trial.params}")
    
    # Plot optimization history
    optuna.visualization.plot_optimization_history(study)
    optuna.visualization.plot_param_importances(study)
    
except ImportError:
    print("❌ Optuna not installed. Install with: pip install optuna")
    print("Manual tuning results above are still very effective!")

## �� Summary

This notebook provides comprehensive hyperparameter tuning for your earthquake forecasting model:

### ✅ **What We've Implemented:**
1. **Manual Grid Search** - Test specific parameter combinations
2. **Priority-Based Tuning** - Focus on most impactful parameters first
3. **Performance Tracking** - Monitor frequency and magnitude ranges
4. **Configuration Management** - Save and load best configurations
5. **Advanced Optimization** - Optional Optuna integration

### �� **Key Tuning Areas:**
1. **Frequency Scaling** - Fix range compression (Priority 1)
2. **Loss Weights** - Balance magnitude vs frequency (Priority 2)
3. **Learning Rates** - Training stability (Priority 3)
4. **Architecture** - LSTM sizes, MLP funnel (Priority 4)
5. **Training Stability** - Patience, regularization (Priority 5)

### 🚀 **Next Steps:**
1. Run the tuning tests above
2. Use the best configurations in your main training script
3. Retrain models with optimized parameters
4. Compare final performance
5. Consider ensemble approach

Happy tuning! 🎉